# Trie

A trie, also called prefix tree, is a kind of search tree. All the descendants of a node have a common prefix of the string associated with that node, and the root is associated with the empty string.

In [47]:
from IPython.display import IFrame
IFrame('https://en.wikipedia.org/wiki/Trie', width=800, height=350)

In [16]:
import collections

AllAppAccount = collections.namedtuple('AllAppAccount', ["app_id"])
AllAppActivity = collections.namedtuple('AllAppActivity', ["app_id"])
AllAppEvent = collections.namedtuple('AllAppEvent', ["app_id"])
AllAchievement = collections.namedtuple('AllAchievement', [])
Achievement = collections.namedtuple('Achievement', ["achievement_id"])
AllAchievementType = collections.namedtuple('AllAchievementType', [])
AllUser = collections.namedtuple('AllUser', [])
User = collections.namedtuple('User', ["user_id"])
AllUserFriend = collections.namedtuple('AllUserFriend', ["user_id"])
AllPost = collections.namedtuple('AllPost', [])
Post = collections.namedtuple('Post', ["post_id"])
AllPostLike = collections.namedtuple('AllPostLike', ["post_id"])

## Build Trie

In [35]:

import networkx as nx
from networkx.utils import generate_unique_node

DYNAMIC = 'dynamic'
SEPERATOR = '/'

def normal_traverse(trie, node, key, params=[]):
    return trie.successor(node, key)

def seperator_traverse(trie, node, key, params):
    dynamic_node = trie.successor(node, DYNAMIC)
    if dynamic_node is not None:
        params.append(key)
        return dynamic_node
    else:
        return normal_traverse(trie, node, key)

def dynamic_traverse(trie, node, key, params):
    if key == SEPERATOR:
        return normal_traverse(trie, node, key)
    else:
        params[-1] += key
        return node

class Trie:
    def __init__(self, graph = nx.DiGraph()):
        self.graph = graph
        self._root = self.__add_node(None, None)

    def search(self, path):
        current = self._root
        params = []
        for char in path.rstrip(SEPERATOR):
            next_node = self.__traverse(current, char, params)
            if next_node is None:
                break
            current = next_node
        return (current, params)

    def insert(self, path_segments, value):
        current = self._root
        for segment in path_segments:
            for char in segment:
                next_node = self.successor(current, char)
                if next_node is None:
                    next_node = self.__add_node(current, char)
                current = next_node
        self.node(current)['clazz'] = value

    def successor(self, node, key):
        return next(filter(lambda n: self.node(n).get('key') == key, self.graph.succ[node]), None)

    def node(self, node):
        return self.graph.nodes[node]

    def __add_node(self, node, key):
        next_node, next_node_attr = self.__build_node(key)
        self.graph.add_node(next_node, **next_node_attr)
        if node is not None:
            self.graph.add_edge(node, next_node)
        return next_node

    def __build_node(self, key, clazz=None):
        return (generate_unique_node(), dict(key=key, clazz=clazz))

    def __traverse(self, node, char, params):
        NODES = {
            DYNAMIC: dynamic_traverse,
            SEPERATOR: seperator_traverse
        }
        return NODES.get(self.node(node).get('key'), normal_traverse)(self, node, char, params)


In [36]:
G = nx.DiGraph()
trie = Trie(G)

## Visualize Trie
### Build graph by Graphviz

In [37]:
import pygraphviz as pgv

def build_graphviz(V, E):
    H = pgv.AGraph(strict=True, directed=False)
    H.add_nodes_from(V)
    H.add_edges_from(E)
    H.layout(prog='dot')
    return H

### Transfer Graphviz to Plotly

In [43]:
import plotly.graph_objs as go
import numpy as np
from ast import literal_eval

def plotly_nodes(G, H, function):
    nodes = list(filter(function, H.nodes()))
    labels = [G.node[v]['key'] for v in nodes]
    hoverlabels = [G.node[v]['clazz'] for v in nodes]
    
    pos = np.array([literal_eval(n.attr['pos']) for n in nodes])

    N = len(pos)
    Xn = [pos[k][0] for k in range(N)]
    Yn = [pos[k][1] for k in range(N)]

    return Xn, Yn, labels, hoverlabels

def plotly_edges(G, H, edges):
    Xe = []
    Ye = []
    for e in edges:
        e0 = np.array(literal_eval(H.get_node(e[0]).attr['pos']))
        e1 = np.array(literal_eval(H.get_node(e[1]).attr['pos']))
        Xe += [e0[0],e1[0], None]
        Ye += [e0[1],e1[1], None]
        
    return Xe, Ye

def edge_scatter(go, Xe, Ye, color='rgb(160,160,160)'):
    return go.Scatter(x=Xe, y=Ye, mode='lines', line=dict(color=color, width=2), hoverinfo='none')

def node_scatter(go, Xn, Yn, color, labels, hoverlabels):
    return go.Scatter(x=Xn, y=Yn, mode='markers+text', text=labels, name='', hoverinfo='none',
                   marker=dict(color=color, size=18, line=dict(color='rgb(100,100,100)', width=1)))

def plotly_node_scatter(go, G, H, function, color):
    Xn, Yn, labels, hoverlabels = plotly_nodes(G, H, function)
    return node_scatter(go, Xn, Yn, color, labels, hoverlabels)

def plotly_edge_scatter(go, G, H, edges, color='rgb(160,160,160)'):
    Xe, Ye = plotly_edges(G, H, edges)
    return edge_scatter(go, Xe, Ye, color)

def plotly_figure(go, G, H, data=[]):
    edges = plotly_edge_scatter(go, G, H, H.edges())
    root_node = plotly_node_scatter(go, G, H, lambda n: G.node[n]['key'] is None, 'rgb(100,100,100)')
    dynamic_nodes = plotly_node_scatter(go, G, H, lambda n: G.node[n]['key'] == DYNAMIC, 'rgb(101,214,67)')
    seperate_nodes = plotly_node_scatter(go, G, H, lambda n: G.node[n]['key'] == SEPERATOR, 'rgb(247,185,43)')
    normal_nodes = plotly_node_scatter(go, G, H, lambda n: G.node[n]['key'] not in [None, DYNAMIC, SEPERATOR], 'rgb(24,140,216)')
    
    axis = dict(showline=False, zeroline=False, showgrid=False, showticklabels=False)
    layout = go.Layout(title="Trie", width=1000, height=800, font_size=15, showlegend=False, xaxis=axis, yaxis=axis,
                       plot_bgcolor='rgb(248,248,248)', margin=dict(l=40, r=40, b=85, t=100), hovermode='closest')
    return go.FigureWidget(data=[edges] + data + [root_node, dynamic_nodes, seperate_nodes, normal_nodes], layout=layout)

def animation(graph):
    plotly_figure(go, graph, build_graphviz(graph.nodes(), graph.edges())).show()

In [44]:
from functools import wraps

def insert_animation(graph, func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        animation(graph.copy())
    return wrapper

insert = insert_animation(G, trie.insert)

In [45]:
insert(["/apps/", [DYNAMIC], "/accounts"], AllAppAccount)
insert(["/apps/", [DYNAMIC], "/activities"], AllAppActivity)
insert(["/apps/", [DYNAMIC], "/events"], AllAppEvent)
insert(["/achievements"], AllAchievement)
insert(["/achievements/", [DYNAMIC]], Achievement)
insert(["/achievement-types"], AllAchievementType)
insert(["/users"], AllUser)
insert(["/users/", [DYNAMIC]], User)
insert(["/users/", [DYNAMIC], "/friends"], AllUserFriend)
insert(["/posts"], AllPost)
insert(["/posts/", [DYNAMIC]], Post)
insert(["/posts/", [DYNAMIC], "/likes"], AllPostLike)

In [50]:
from itertools import tee

def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

def path_animation(graph, root, destination):
    paths = nx.shortest_path(graph, source=root, target=destination)
    H = build_graphviz(graph.nodes(), graph.edges())   
    path = plotly_edge_scatter(go, graph, H, pairwise(paths), 'red')    
    fig = plotly_figure(go, graph, H, [path])
    fig.show()
    
def search_animation(trie, func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        destination, params = func(*args, **kwargs)
        path_animation(trie.graph.copy(), trie._root, destination)
    return wrapper

search = search_animation(trie, trie.search)

In [52]:
search("/users")

In [53]:
search('/anything')